In [ ]:
# Import required libraries
import sys
import os
import pandas as pd
import numpy as np
import duckdb
import logging
from pathlib import Path
from datetime import datetime, date, timedelta

# Add project root to Python path
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Import custom modules
from config import config
from utils.database import DuckDBManager
from utils.multi_exchange_loader import MultiExchangeLoader

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ Imports successful!")


In [ ]:
# Create a new database for the multi-exchange setup
multi_exchange_db_path = './multi_exchange_data_lake.duckdb'

# Initialize database manager with new database
db_manager = DuckDBManager(database_path=multi_exchange_db_path)
conn = db_manager.connect()

print(f"✅ Connected to new database: {multi_exchange_db_path}")
print(f"DuckDB Version: {conn.execute('SELECT version()').fetchone()[0]}")


In [ ]:
# Create the multi-exchange schema
print("🏗️ Creating database schema...")

# Read the SQL schema file
with open('create_tables_with_partitioning.sql', 'r') as f:
    sql_script = f.read()

# Split into individual statements and execute
statements = [stmt.strip() for stmt in sql_script.split(';') if stmt.strip() and not stmt.strip().startswith('--')]

successful_statements = 0
failed_statements = 0

for i, statement in enumerate(statements):
    if statement:
        try:
            conn.execute(statement)
            successful_statements += 1
            if i % 10 == 0:  # Progress indicator
                print(f"  ✓ Executed {i+1}/{len(statements)} statements...")
        except Exception as e:
            failed_statements += 1
            if "already exists" not in str(e).lower():
                print(f"  ⚠️  Warning in statement {i+1}: {e}")

print(f"\n✅ Schema creation completed!")
print(f"   Successful: {successful_statements}")
print(f"   Failed/Warnings: {failed_statements}")


In [ ]:
# Initialize the multi-exchange loader
loader = MultiExchangeLoader(db_manager=db_manager)

print("✅ Multi-Exchange Loader initialized")
print("\n📋 Supported Exchanges:")
for exchange, config in loader.exchanges.items():
    print(f"  🏢 {exchange}:")
    print(f"     Stages: {', '.join(config['stages'])}")
    print(f"     Table: {config['table']}")
    print(f"     Timezone: {config['timezone']}")
